# Домашняя работа

В этой работы мы будем учиться предсказывать зарплату data scientist-ов в зависимочти от ряда факторов с помощью градиентоного бустинга

Цель домашней работы научиться предсказывать зарплаты (salary_in_usd) по ряды факторов

work_year: The number of years of work experience in the field of data science.

experience_level: The level of experience, such as Junior, Senior, or Lead.

employment_type: The type of employment, such as Full-time or Contract.

job_title: The specific job title or role, such as Data Analyst or Data Scientist.

salary: The salary amount for the given job.

salary_currency: The currency in which the salary is denoted.

salary_in_usd: The equivalent salary amount converted to US dollars (USD) for comparison purposes.

employee_residence: The country or region where the employee resides.

remote_ratio: The percentage of remote work offered in the job.

company_location: The location of the company or organization.

company_size: The company’s size is categorized as Small, Medium, or Large.

In [ ]:
!wget https://raw.githubusercontent.com/hse-ds/iad-intro-ds/master/2023/homeworks/hw-08-boosting/ds_salaries.csv

In [ ]:
import pandas as pd

In [39]:
df = pd.read_csv("ds_salaries.csv")

In [ ]:
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


## Задание 1 (0.5 балла)

Разделите выборку на train, val, test (80%, 10%, 10%) в качестве таргета выберите salary_in_usd, удалите из признаков (salary) чтобы избежать лика в данных.

In [40]:
y = df['salary_in_usd']
data = df.drop(['salary_in_usd', 'salary'], axis=1)

In [42]:
from sklearn.model_selection import train_test_split

X_train, X_test_val, y_train, y_test_val = train_test_split(data, y, train_size=0.8, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=42)

In [ ]:
X_train.shape, X_val.shape, X_test.shape

((3004, 9), (376, 9), (375, 9))

## Задание 2 (0.5 балла)

Обучите модель линейной регресии и оцените её качество через mape и rmse

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error as MAPE
from sklearn.metrics import mean_squared_error as MSE

nested_columns = X_train.columns[X_train.dtypes != object]

lr = LinearRegression().fit(X_train[nested_columns], y_train)
mape_err = MAPE(y_test, lr.predict(X_test[nested_columns]))
rmse_err = MSE(y_test, lr.predict(X_test[nested_columns]), squared=False)

mape_err, rmse_err
# качество ужасное ;с

(0.8028552771954387, 60506.32083408822)

## Задание 3 (0.5 балла)

In [ ]:
!pip install --upgrade xgboost
!pip install --upgrade catboost
!pip install --upgrade lightgbm

Обучите модель бустинга выберите любую из трех библиотек catboost, xgboost, lightgbm и оцените её качество через mape и rmse

In [ ]:
from catboost import CatBoostRegressor

cbr = CatBoostRegressor().fit(X_train, y_train, cat_features=list(X_train.columns[X_train.dtypes == object]), eval_set=(X_val, y_val), verbose=False)
mape_err = MAPE(y_test, cbr.predict(X_test))
rmse_err = MSE(y_test, cbr.predict(X_test), squared=False)

mape_err, rmse_err
# модель примерно в два раза улучшилась
# не идеальная, конечно, но уже неплохо, пойдёт :|

(0.41394340763601023, 46032.48343365925)

## Задание 4 (0.5 балла)

Далее предобработайте категориальные признаки с помощью onehot кодирования, labelencoder и счетчиков (mean target encoding) сравните качество моделей с помощью mape и rmse. Необходимо обучить две модели линейную и бустинг. Какие выводы можно сделать? Выберите лучшую из моделей и дальше будем работать с ней

In [ ]:
# onehot: employment_type  , salary_currency
# label:  experience_level , company_size
# target: job_title        , employee_residence , company_location

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

onehot = OneHotEncoder(drop='first').fit(df[['employment_type', 'salary_currency']])

X_train_coded = pd.DataFrame(data=onehot.transform(X_train[['employment_type', 'salary_currency']]).toarray(), index=X_train.index, columns=onehot.get_feature_names_out())
X_val_coded = pd.DataFrame(data=onehot.transform(X_val[['employment_type', 'salary_currency']]).toarray(), index=X_val.index, columns=onehot.get_feature_names_out())
X_test_coded = pd.DataFrame(data=onehot.transform(X_test[['employment_type', 'salary_currency']]).toarray(), index=X_test.index, columns=onehot.get_feature_names_out())

X_train = pd.concat([X_train.drop(['employment_type', 'salary_currency'], axis=1), X_train_coded], axis=1)
X_val = pd.concat([X_val.drop(['employment_type', 'salary_currency'], axis=1), X_val_coded], axis=1)
X_test = pd.concat([X_test.drop(['employment_type', 'salary_currency'], axis=1), X_test_coded], axis=1)

for column in ['experience_level', 'company_size']:
    label = LabelEncoder().fit(df[column])
    X_train[column] = label.transform(X_train[column])
    X_val[column] = label.transform(X_val[column])
    X_test[column] = label.transform(X_test[column])

for column in ['job_title', 'employee_residence', 'company_location']:
    X_train[column] = X_train.groupby(by=column)[column].transform('count')
    X_val[column] = X_val.groupby(by=column)[column].transform('count')
    X_test[column] = X_test.groupby(by=column)[column].transform('count')

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

lr = LinearRegression().fit(X_train, y_train)
lr_mape_err = MAPE(y_test, lr.predict(X_test))
lr_rmse_err = MSE(y_test, lr.predict(X_test), squared=False)

gbr = GradientBoostingRegressor().fit(X_train, y_train)
gbr_mape_err = MAPE(y_test, gbr.predict(X_test))
gbr_rmse_err = MSE(y_test, gbr.predict(X_test), squared=False)

print(f'Linear MAPE = {lr_mape_err} ; Linear RMSE = {lr_rmse_err}')
print(f'Boosting MAPE = {gbr_mape_err} ; Boosting RMSE = {gbr_rmse_err}')
# Выводы: линейная модель почти в два раза лучше стала, но
# бустинг ещё чуть лучше работает

Linear MAPE = 0.4377851023286178 ; Linear RMSE = 55213.29737607933
Boosting MAPE = 0.3932961557913562 ; Boosting RMSE = 49183.7936933939


## Задание 5 (0.5 балла)

Покажите, где ошибается ваша модель. Выведите топ 20 примеров с наибольшей ошибкой. Проанализируете их, постарайтесь выделить группы с причинами ошибок в моделе. Какие выводы можно сделать? Что стоит изменить в данных чтобы улучшить качество модели?

In [ ]:
import numpy as np

y_pred = gbr.predict(X_test)
err = np.abs(y_pred - y_test)
_, indexes = zip(*sorted(zip(err.values, err.index), reverse=True)[:20])
pd.DataFrame(df, index=indexes, columns=df.columns)
# сложно выделить группы, больше похоже на какие-то выбросы (а также странности)
# индекс 2589 -- Product Data Scientist 2022 года получает нищенские 8к ...
# индексы 3387 и 3084 -- два одинаковых Data Engineer'а, но один на удалёнке получает зп в 10 раз меньше, чем тот, который батрачит в офисе (strange...)
# НО индексы 358 и 1425 -- два одинаковых Machine Learning Engineer'а, но тот, кто на удалёнке получает на 30к больше (к предыдущим двух типам в копилку strange...)
# индекс 3037 -- Data Architect, если брать остальных с такими же характеристиками, кроме job_title, то он получает зп примерно в 5 раз меньше, хотя Data Architect...
# КОРОЧЕ, очень стойкое ощущение, что это выбросы
# но появилось ощущение, что стоит предобработать remote_ratio каким-нибудь scaler'ом (звучит как фигня, но вдруг...)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
478,2023,EX,FT,Director of Data Science,353200,USD,353200,US,0,US,M
1105,2023,SE,FT,Data Scientist,370000,USD,370000,US,0,US,M
3609,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2281,2022,SE,FT,Data Science Manager,299500,USD,299500,US,0,US,M
2009,2022,SE,FT,Machine Learning Engineer,255000,USD,255000,MX,100,MX,M
358,2023,SE,FT,Machine Learning Engineer,304000,USD,304000,US,100,US,M
3387,2022,SE,FT,Data Engineer,25000,USD,25000,US,100,US,M
58,2023,EX,FT,Data Scientist,258750,USD,258750,US,0,US,M
3519,2021,EX,FT,Head of Data,230000,USD,230000,RU,50,RU,L
549,2023,SE,FT,Data Scientist,275300,USD,275300,US,100,US,M


## Задание 6 (0.5 балла)

Придумайте признаки для улучшения качества модели на основе предыдущего пункта. Как вам кажется какими признаками можно улучшить качетсво модели? Реализуйте признаки и проверьте улучшилось ли качество модели. Полный бал за задание ставится даже при отсутствии улучшения качества модели, важно попробовать проверить свои гипотезы и сделать выводы

In [ ]:
# work_year -- предобработать относительно 2023 года (2023 - work_year)
# experience_level -- labelencoder
# employment_type -- onehot
# job_title -- onehot
# salary -- удалить
# salary_currency -- удалить, так как считаю данный признак бесполезным
# salary_in_usd -- целевая переменная
# employee_residence и company_location -- объединить по принципу соответствия (тип, employee_residence == company_location), т.е. преобразование в {0, 1}
# remote_ratio -- посмотрел на него ещё раз и понял, что данный признак принимает только три значения (0, 50, 100), поэтому хочу попробовать onehot
# company_size -- labelencoder

In [ ]:
df_new = pd.DataFrame()

df_new['work_year'] = 2023 - df['work_year']

df_new['experience_level'] = LabelEncoder().fit_transform(df['experience_level'])

df_new['residence_and_location'] = pd.DataFrame(df['employee_residence'] == df['company_location'], dtype=int)

df_new['company_size'] = LabelEncoder().fit_transform(df['company_size'])

df_new = df_new.join(pd.DataFrame(OneHotEncoder(drop='first').fit_transform(df[['employment_type', 'job_title', 'remote_ratio']]).toarray()))

y = df['salary_in_usd']
df_new.columns = df_new.columns.astype(str)

In [ ]:
df_new.head()

,work_year,experience_level,residence_and_location,company_size,0,1,2,3,4,5,...,87,88,89,90,91,92,93,94,95,96
0,0,3,1,0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0,2,1,2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0,2,1,2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0,3,1,1,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0,3,1,1,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
X_train, X_test_val, y_train, y_test_val = train_test_split(df_new, y, train_size=0.8, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=42)

X_train.shape, X_val.shape, X_test.shape

((3004, 101), (376, 101), (375, 101))

In [ ]:
gbr = GradientBoostingRegressor().fit(X_train, y_train)
gbr_mape_err = MAPE(y_test, gbr.predict(X_test))
gbr_rmse_err = MSE(y_test, gbr.predict(X_test), squared=False)

print(f'Boosting MAPE = {gbr_mape_err} ; Boosting RMSE = {gbr_rmse_err}')
# ухудшилось :(
# видимо, слишком перемудрил
# обидно, досадно

Boosting MAPE = 0.5995532705928217 ; Boosting RMSE = 50709.802953390514


## Задание 7 (0.5 балла)

Теперь используете исключительно catboost. Обучить модель. Подберите оптимальные гиперпараметры. Используйте pool для передачи данных в модель с указанием какие признаки категориальные, а какие нет с помощью параметры cat_features. Оцените качество итоговой модели, скорость обучения и скорость предсказания

In [ ]:
!pip install --upgrade tqdm

In [ ]:
from tqdm import tqdm

In [ ]:
iterations = [100, 500, 1000, 10000]
learning_rate = [0.005, 0.01, 0.03, 0.05, 0.1, 0.3]
depth = [6, 7, 8, 9, 10]

gipers = []
for i in iterations:
    for j in learning_rate:
        for k in depth:
            gipers.append((i, j, k))

X_train, X_test_val, y_train, y_test_val = train_test_split(data, y, train_size=0.8, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=42)

In [45]:
# 3 часа подбиралось..
# результат не очень, но пойдёт

from catboost import CatBoostRegressor
from catboost import Pool

errs = []
for i, j, k in tqdm(gipers):
    cbr = CatBoostRegressor(iterations=i, learning_rate=j, depth=k, random_state=42, verbose=False).fit(Pool(data=X_train, label=y_train, cat_features=list(X_train.columns[X_train.dtypes == object])), eval_set=(X_val, y_val), verbose=False)
    mape_err = MAPE(y_test, cbr.predict(X_test))
    rmse_err = MSE(y_test, cbr.predict(X_test), squared=False)
    print(mape_err, rmse_err, i, j, k)
    errs.append((mape_err, rmse_err, i, j, k))

  0%|          | 1/200 [00:00<00:59,  3.36it/s]

0.8319681916451351 59741.49197123331 100 0.001 6


  1%|          | 2/200 [00:00<01:08,  2.88it/s]

0.832232309503328 59751.479313486416 100 0.001 7


  2%|▏         | 3/200 [00:01<01:16,  2.58it/s]

...

0.49078491222092036 55360.283668139826 10000 1 8


100%|█████████▉| 199/200 [3:07:54<03:49, 229.79s/it]

0.5045318036239832 55418.82221109371 10000 1 9


100%|██████████| 200/200 [3:14:20<00:00, 58.30s/it] 

0.510329921434833 58088.919577593566 10000 1 10


In [54]:
sorted(errs)[:3]

[(0.4046479879539649, 46418.34872888377, 10000, 0.03, 6),
 (0.4088812267058772, 45333.244607952234, 1000, 0.05, 6),
 (0.4117402345315723, 47296.751712789366, 1000, 0.3, 7)]

In [58]:
cbr = CatBoostRegressor(iterations=1000, learning_rate=0.05, depth=6, random_state=42, verbose=False).fit(Pool(data=X_train, label=y_train, cat_features=list(X_train.columns[X_train.dtypes == object])), verbose=False)
mape_err = MAPE(y_test, cbr.predict(X_test))
rmse_err = MSE(y_test, cbr.predict(X_test), squared=False)
print(mape_err, rmse_err)

0.4088812267058772 45333.244607952234


## Задание 8 (0.5 балла)

Теперь используете исключительно xgboost. Обучить модель. Подберите оптимальные гиперпараметры. Закодируйте категориальные переменные, как хотите. Оцените качество итоговой модели, скорость обучения и скорость предсказания.

## Задание 9 (0.5 балла)

Теперь используете исключительно lightgbm. Обучить модель. Подберите оптимальные гиперпараметры. Закодируйте категориальные переменные, как хотите. Оцените качество итоговой модели, скорость обучения и скорость предсказания.

## Задание 10 (0.5 балла)

Сделайте выводы про модели бустинга, какая из моделей показала лучший результат по качеству, скорости обучения и скорости предсказания